# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [2]:
import sqlite3
import pandas as pd
conn = sqlite3.Connection('data.sqlite')
cur = conn.cursor()

## Select the Average Number of Orders on a Product by Product Basis

In [6]:
#Your code here; use a subquery. No join will be necessary.
cur.execute("""select AVG(numOrders)
                    from (select productCode,
                                 count(orderNumber) as numOrders
                                 from orderdetails
                                 group by 1
                         );
          """
         )
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,AVG(numOrders)
0,27.486239


## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [16]:
#Your code here; use a subquery. No join will be necessary.
cur.execute("""select customerNumber, contactLastName, contactFirstName
                    from customers
                    where customerNumber in (select customerNumber from orders where orderDate = '2003-01-31');
            """
           )
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [18]:
cur.execute("""select productName, 
                      count(orderNumber) as numberOrders,
                      sum(quantityOrdered) as total_num_sold
                      from products
                      join orderdetails
                      using(productcode)
                      group by 1
                      order by total_num_sold desc;
            """
           )
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,productName,numberOrders,total_num_sold
0,1992 Ferrari 360 Spider red,53,1808
1,1937 Lincoln Berline,28,1111
2,American Airlines: MD-11S,28,1085
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,1930 Buick Marquette Phaeton,28,1074


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [20]:
cur.execute("""select productName, 
                      count(distinct customerNumber) total_buyers
                      from products
                      join orderdetails
                      using(productcode)
                      join orders
                      using(ordernumber)
                      group by 1
                      order by total_buyers desc;
            """
           )
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,productName,total_buyers
0,1992 Ferrari 360 Spider red,40
1,1934 Ford V8 Coupe,27
2,1952 Alpine Renault 1300,27
3,1972 Alfa Romeo GTA,27
4,Boeing X-32A JSF,27


## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [31]:
cur.execute("""select distinct employeeNumber, 
                      officeCode,
                      o.city,
                      lastname,
                      firstname
                      from employees e
                      join offices o
                      using (officeCode)
                      join customers c
                      on e.employeeNumber = c.salesRepEmployeeNumber
                      join orders 
                      using(customerNumber)
                      join orderdetails
                      using(orderNumber)
                      where productCode in (select productCode
                                                   from products p
                                                   join orderdetails od
                                                   using(productCode)
                                                   join orders
                                                   using(orderNumber)
                                                   group by 1
                                                   having count(distinct customerNumber) <= 20
                                            );
            """
           )
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,officeCode,city,lastName,firstName
0,1370,4,Paris,Hernandez,Gerard
1,1401,4,Paris,Castillo,Pamela
2,1621,5,Tokyo,Nishi,Mami
3,1501,7,London,Bott,Larry
4,1337,4,Paris,Bondur,Loui
5,1286,3,NYC,Tseng,Foon Yue
6,1166,1,San Francisco,Thompson,Leslie
7,1165,1,San Francisco,Jennings,Leslie
8,1612,6,Sydney,Marsh,Peter
9,1188,2,Boston,Firrelli,Julie


## Select the Employee Number, Name (First and Last) and Number of Customers --> for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [38]:
cur.execute("""select distinct employeeNumber, 
                      lastName,
                      firstName,
                      count(distinct customerNumber) as num_of_customers
                      from employees e
                      join customers c
                      on e.employeeNumber = c.salesRepEmployeeNumber
                      group by 1
                      having avg(creditlimit) > 15000;
            """
           )
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,lastName,firstName,num_of_customers
0,1165,Jennings,Leslie,6
1,1166,Thompson,Leslie,6
2,1188,Firrelli,Julie,6
3,1216,Patterson,Steve,6
4,1286,Tseng,Foon Yue,7
5,1323,Vanauf,George,8
6,1337,Bondur,Loui,6
7,1370,Hernandez,Gerard,7
8,1401,Castillo,Pamela,10
9,1501,Bott,Larry,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!